In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go

# Import charging data, plot it, and analyze it

In [ ]:
# Insert current working directory
admin_shark = pd.read_hdf(r'/home/lencon/Git_Research/SIGI_LAPTOP_TRANSFER/Documents/Research/SIGI_Research_Meetings/19_apr_2023/Full Year/admin_shark_03_01_2022_to_03_01_2023.h5')

In [ ]:
admin_shark

In [ ]:
# CE-CERT Fast Charger
l3_power = admin_shark[admin_shark.tag == 'EV Charger L3 Watts, 3-Ph total'].reset_index(drop=True)
l3_power

In [ ]:
# The 4 regular CE-CERT EV Chargers
l2_power = admin_shark[admin_shark.tag == 'EV Charger L2 Watts, 3-Ph total'].reset_index(drop=True)
l2_power

In [ ]:
l2_power.value.describe()

In [ ]:
# Air Conditioning Loads of CE-CERT Admin buling
hvac_power = admin_shark[admin_shark.tag == 'T1 (HVAC) Watts, 3-Ph total'].reset_index(drop=True)
hvac_power 

In [ ]:
# 120 and 240 V outlets at CE-CERT Admin Builiding
plug_load = admin_shark[admin_shark.tag == 'T2 (Plug Loads) Watts, 3-Ph total'].reset_index(drop=True)
plug_load

## Function to plot data as html, please verify you have plotly installed

In [22]:
import plotly
import plotly.graph_objects as go
def plot_graph(x_data, y_data, line_name, title, x_title, y_title, filename):
    # Define the figure
    fig = go.Figure()
    # Add data to figure
    for i in range(0, len(x_data), 1):
        fig.add_trace(go.Scattergl(x=x_data[i], y=y_data[i], name= line_name[i]))
            
    # Create Layout for the Html
    
    fig.update_layout(
        title=go.layout.Title(
            text= title,
            xref="paper",
            x=0.5
        ),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text=x_title,
            ),
            rangeslider=dict(
                visible=True
            )
        ),
        yaxis=go.layout.YAxis(
            title=go.layout.yaxis.Title(
                text=y_title,
            )
        ),
        plot_bgcolor = "white"
    )
    fig.show()
    fig.write_html(filename + '.html')
    # Define the figure
    fig = go.Figure()
    # Add data to figure
    for i in range(0, len(x_data), 1):
        fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], name= line_name[i]))
            
    # Create Layout for the Html
    
    fig.update_layout(
        title=go.layout.Title(
            text= title,
            xref="paper",
            x=0.5
        ),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text=x_title,
            ),
            rangeslider=dict(
                visible=False
            )
        ),
        yaxis=go.layout.YAxis(
            title=go.layout.yaxis.Title(
                text=y_title,
            )
        ),
        plot_bgcolor = "white"
    )
    fig.update_layout(
    font=dict(
        family="sans-serif",
        size=20,  # Set the font size here
    ),
    legend=dict(
        x=0.5,
        y=1,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=20,
            color="black"
            )
        )
    )
    fig.write_image(filename + '.pdf', engine = 'orca')
    fig.write_image(filename + '.pdf', engine = 'orca')

Plot entire interval

In [ ]:
plot_graph([l2_power.time], [l2_power.value], "Level 2", "Level 2 Sharkmeter Data", "Time", "Power (kW)", "L2")

Plot a couple of days

In [ ]:
plot_graph([l2_power.time[l2_power.time.between('06/03/2022', '06/12/2022')]], [l2_power.value[l2_power.time.between('06/03/2022', '06/12/2022')]], "Level 2", "Level 2 Sharkmeter Data", "Time", "Power (kW)", "L2_short")

## Function to average a large interval into one average day data
### Used to determine typical data

In [9]:
def day_time(freq = '1min'):
    # Create the number on intervals during the day
    time_array = pd.date_range('0:00', '23:59:59', freq=freq).strftime('%H:%M')
    return(time_array)

def time_sep_label(df, time_column = 'time', resample_rate = '1T', value_column = 'value'):
    # Ensure time column is in datetime format
    df[time_column] = pd.to_datetime(df[time_column])
    # Set time column as index
    df = df.set_index(df[time_column])
    # Remove time column from dataframe since it is now the index
    df = df.drop(columns = [time_column])
    # Clean up and resample missing data 
    df = df.resample(resample_rate).mean()
    # Extract the time period ths user wants in a day as an integer
    resample_rate_int =  int(resample_rate.rstrip(resample_rate[-1]))
    # Change '1T' to '1min' Call the daytime function to produce the time intervals
    time_array = day_time(str(resample_rate_int) + 'min')
    # Create a new column for labeling each row 
    df['time_interval_label'] = np.nan
    # Declare the list for indexes for ever interval
    avg_day = []
    # Extract the indexes for ever interval
    for i in range(len(time_array) - 1):
        avg_day.append(df[value_column].between_time(time_array[i], time_array[i + 1], inclusive = 'left').mean())
    return[time_array, avg_day]

def time_sep_label_sum(df, time_column = 'time', resample_rate = '1T', value_column = 'value'):
    # Ensure time column is in datetime format
    df[time_column] = pd.to_datetime(df[time_column])
    # Set time column as index
    df = df.set_index(df[time_column])
    # Remove time column from dataframe since it is now the index
    df = df.drop(columns = [time_column])
    # Clean up and resample missing data 
    df = df.resample(resample_rate).mean()
    # Extract the time period ths user wants in a day as an integer
    resample_rate_int =  int(resample_rate.rstrip(resample_rate[-1]))
    # Change '1T' to '1min' Call the daytime function to produce the time intervals
    time_array = day_time(str(resample_rate_int) + 'min')
    # Create a new column for labeling each row 
    df['time_interval_label'] = np.nan
    # Declare the list for indexes for ever interval
    avg_day = []
    # Extract the indexes for ever interval
    for i in range(len(time_array) - 1):
        avg_day.append(len(df[value_column].between_time(time_array[i], time_array[i + 1], inclusive = 'left').dropna()))
    avg_day.append(len(df[value_column].between_time(time_array[-1], time_array[0], inclusive = 'left').dropna()))
    return[time_array, avg_day]

Call function to average the interval for the regular EV loads

In [ ]:
time_array, avg_day = time_sep_label(l2_power)

Create a dataframe for the plot and reove the last row

In [ ]:
avg_day = pd.DataFrame({'time': time_array[0:-1], 'avg_day': avg_day})
avg_day

Plot the average day with time

In [ ]:
plot_graph([avg_day.time], [avg_day.avg_day / 1000], 'Level 2 Average Power', 'Lever 2 Chargers Average Load', 'time', 'Power (kW)', 'l2_avg_day')

Plot the average day with minutes in a day (1440 total)

In [ ]:
plot_graph([avg_day.index], [avg_day.avg_day / 1000], 'Level 2 Average Power', 'Lever 2 Chargers Average Load', 'time', 'Power (kW)', 'l2_avg_day_int')

## Fit the the data using SciPy

In [ ]:
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt

#data = np.loadtxt('data.txt', delimiter=',')
x, y = avg_day.index, (avg_day.avg_day / 1000)

plt.plot(x,y)
plt.show()

def func(x, *params):
    y = np.zeros_like(x)
    for i in range(0, len(params), 3):
        ctr = params[i]
        amp = params[i+1]
        wid = params[i+2]
        y = y + amp * np.exp( -((x - ctr)/wid)**2)
    return y

guess = [558, 5.87, 332, 872, 3.17, 358, 1235, 1.38, 300]
#for i in range(3):
#    guess += [60+80*i, 46000, 25]   

popt, pcov = curve_fit(func, x, y, p0=guess, maxfev=5000)
print (popt)
print ("-----------------------------------------------------------------")
print (pcov)
fit = func(x, *popt)

plt.plot(x, y)
plt.plot(x, fit , 'r-')
plt.show()

# Function to generate Random EV arrival times using a compound Gaussian

In [ ]:
import numpy as np
import pandas as pd
# Creates the Gaussian for number of random arrivals for a day (Scalar value)
def number_of_arrivals(mean, std_dev, size =1):
    num_arriv = np.array([])
    for i in range(len(mean)):
        num_arriv = np.append(num_arriv, np.random.default_rng().normal(mean[i], std_dev[i], size))
    num_arriv = np.sort(np.absolute(num_arriv))
    return (num_arriv)
# Creates a compund Gaussian with user imputed size (vector length) 
# A vector is expected for for the function fir both the mean and std dev
# [1 ... N] creates an Nth compound Gaussian eg, mean = [10, 20, 30], std_dev = [1, 2, 3]  creates a tri-Gaussian
def compound_gauss(mean, std_dev, size):
    if len(mean) == len(std_dev):
        cpd_gauss = np.array([])
        for i in range(len(mean)):
            if size[i] <= 1:
                size[i] = 1
            cpd_gauss = np.append(cpd_gauss, np.random.default_rng().normal(mean[i], std_dev[i], int(size[i])))
        #print(cpd_gauss)
        cpd_gauss = np.sort(cpd_gauss)
    return (cpd_gauss)
# Creates the Guassian for arrival and leave times in minutes in a day
def random_ev_arrivals_times(arriv_mean, arriv_std_dev, cpd_mean, cpd_std_dev, charge_mean, charge_std_dev):
    # Size (vector length) is determined by number_of_arrivals function
    size = number_of_arrivals(arriv_mean, arriv_std_dev)
    # Input the time when peaks occur
    arriv_time = compound_gauss(cpd_mean, cpd_std_dev, size)
    print(arriv_time)
    # Input the average duration of charging for each peak 
    leave_time = compound_gauss(charge_mean, charge_std_dev, size)
    return [arriv_time, leave_time]
# Creates a datafrrame with the arrival times, and adds a time column 
# If multiple charging sessions occur they add up
def date_time_arrival_times(start_date, num_days, arriv_mean, arriv_std_dev, cpd_mean, cpd_std_dev, charge_mean, charge_std_dev):
    # Create a dataframe to be populated 
    df = pd.DataFrame(columns = ['arriv_time', 'leave_time'])
    # Timestamp for each day counter 
    current_date = start_date
    # Populates the dataframe with random arrival for each day
    # Each day is added one at a time in the for loop
    for i in range(num_days):
        # Create temporary dataframe for that day that will be added main dataframe
        temp = pd.DataFrame(columns = ['arriv_time'])
        # Call the random arrival time function for each day
        arriv_time, leave_time = random_ev_arrivals_times(arriv_mean, arriv_std_dev, cpd_mean, cpd_std_dev, charge_mean, charge_std_dev)
        # Converts random arrival times (minutes in a day) into timedeltas which are added to that day's timestamp
        # Converts random arrival times into pandas timestamps 
        temp['arriv_time'] = current_date + pd.to_timedelta(arriv_time, unit = 'min')
        # Converts random leave times (duration) into timestamps by adding arrival time to leave time timedeltas
        temp['leave_time'] = temp['arriv_time'] + pd.to_timedelta(leave_time, unit = 'min')
        # Add daily dataframe to the main dataframe
        df = pd.concat([df,temp])
        # Update the current day timestamp by adding one day
        current_date = current_date + pd.Timedelta(1, "d")
    # Ensures that arrival times are in order
    df = df.sort_values(by=['arriv_time'])
    # Reset dataframe to organized arrival times
    df = df.reset_index(drop = True)
    # Seperate arrival times and leave times to create charging sessions 
    df1 = df[['arriv_time']]
    # Set a charging session to 1
    df1['value'] = 1
    df1['time'] = df1['arriv_time']
    df2 = df[['leave_time']]
    # Set a charging session to -1 to negate a charging to shut it off
    df2['value'] = -1
    df2['time'] = df2['leave_time']
    # Combine the seperated dataframes to create a charging sessions 
    ev = pd.concat([df1, df2])
    # Sort the combined dataframes, by time 
    ev = ev.sort_values(by=['time'])
    # Reset dataframe to organized times
    ev = ev.reset_index(drop = True)
    # Adds the 1 and -1s to get number of charging sessions occuring at every time interval
    ev['fuzz'] = ev.value.cumsum()
    # 0 - 4 charging session can occur since there are 4 charging stations
    ev['fuzz'] = ev['fuzz'].clip(lower=0, upper = 4)
    # Multiply by 5 kW which is the power consumption for each EV charger
    ev['charge'] = ev['fuzz'] * 5 
    # Organize columns 
    ev = ev[['time', 'value', 'fuzz', 'charge']]
    return [df, ev]

## Call the function and adjust the parameters

In [ ]:
arriv_mean = [6, 2, 1] # Average number of car arrivals
arriv_std_dev = [4, 2, 0.3] # Car arrivals standard deviation
cpd_mean = [558, 872, 1235] # Arrival times peaks 
cpd_std_dev = [332, 358, 300] # Arrival times standard deviation
ev_arriv = np.array([])
start_date = pd.Timestamp(2023, 1, 1, 0) # Start date for the time deltas
charge_mean = [90, 90, 90] # Average length of charging session 
charge_std_dev = [30, 30, 30] # Standard deviation of charging length
num_days = 365 # Number of outputed random data 
# Call the function 
df, ev = date_time_arrival_times(start_date, num_days, arriv_mean, arriv_std_dev,
                              cpd_mean, cpd_std_dev, charge_mean, charge_std_dev)
# Display dataframe
ev

# Function to generate Random EV arrival times using a compound Gaussian and a Poisson Functions 

In [4]:
import numpy as np
import pandas as pd
# Creates the Poisson for number of random arrivals for a day (Scalar value)
def number_of_arrivals(mean, size =1):
    num_arriv = np.array([])
    for i in range(len(mean)):
        num_arriv = np.append(num_arriv, np.random.poisson(mean[i], size))
    num_arriv = np.sort(np.absolute(num_arriv))
    return (num_arriv)
# Creates a compund Gaussian with user imputed size (vector length) 
# A vector is expected for for the function fir both the mean and std dev
# [1 ... N] creates an Nth compound Gaussian eg, mean = [10, 20, 30], std_dev = [1, 2, 3]  creates a tri-Gaussian
def compound_gauss(mean, std_dev, size):
    if len(mean) == len(std_dev):
        cpd_gauss = np.array([])
        for i in range(len(mean)):
            if size[i] <= 1:
                size[i] = 1
            cpd_gauss = np.append(cpd_gauss, np.random.default_rng().normal(mean[i], std_dev[i], int(size[i])))
        cpd_gauss = np.sort(cpd_gauss)
    return (cpd_gauss)
def compound_poisson(mean, size):
    cpd_poisson = np.array([])
    for i in range(len(mean)):
        if size[i] <= 1:
            size[i] = 1
        cpd_poisson = np.append(cpd_poisson, np.random.poisson(mean[i], int(size[i])) * 60  + np.random.randint(-30, high=30, size = int(size[i])))
    cpd_poisson = np.sort(cpd_poisson)
    return (cpd_poisson)
# Creates the Poisson for arrival and leave times in minutes in a day
def random_ev_arrivals_times(arriv_mean, cpd_mean, cpd_std_dev, charge_mean, charge_std_dev):
    # Size (vector length) is determined by number_of_arrivals function
    size = number_of_arrivals(arriv_mean)
    # Input the time when peaks occur
    arriv_time = compound_poisson(cpd_mean, size)
    # Input the average duration of charging for each peak 
    leave_time = compound_gauss(charge_mean, charge_std_dev, size)
    return [arriv_time, leave_time]
# Creates a datafrrame with the arrival times, and adds a time column 
# If multiple charging sessions occur they add up
def date_time_arrival_times(start_date, num_days, arriv_mean, cpd_mean, cpd_std_dev, charge_mean, charge_std_dev):
    # Create a dataframe to be populated 
    df = pd.DataFrame(columns = ['arriv_time', 'leave_time'])
    # Timestamp for each day counter 
    current_date = start_date
    # Populates the dataframe with random arrival for each day
    # Each day is added one at a time in the for loop
    for i in range(num_days):
        # Create temporary dataframe for that day that will be added main dataframe
        temp = pd.DataFrame(columns = ['arriv_time'])
        # Call the random arrival time function for each day
        arriv_time, leave_time = random_ev_arrivals_times(arriv_mean, cpd_mean, cpd_std_dev, charge_mean, charge_std_dev)
        # Converts random arrival times (minutes in a day) into timedeltas which are added to that day's timestamp
        # Converts random arrival times into pandas timestamps 
        temp['arriv_time'] = current_date + pd.to_timedelta(arriv_time, unit = 'min')
        # Converts random leave times (duration) into timestamps by adding arrival time to leave time timedeltas
        temp['leave_time'] = temp['arriv_time'] + pd.to_timedelta(leave_time, unit = 'min')
        # Add daily dataframe to the main dataframe
        df = pd.concat([df,temp])
        # Update the current day timestamp by adding one day
        current_date = current_date + pd.Timedelta(1, "d")
    print(len(df.arriv_time))
    # Ensures that arrival times are in order
    df = df.sort_values(by=['arriv_time'])
    # Reset dataframe to organized arrival times
    df = df.reset_index(drop = True)
    # Seperate arrival times and leave times to create charging sessions 
    df1 = df[['arriv_time']]
    # Set a charging session to 1
    df1['value'] = 1
    df1['time'] = df1['arriv_time']
    df2 = df[['leave_time']]
    # Set a charging session to -1 to negate a charging to shut it off
    df2['value'] = -1
    df2['time'] = df2['leave_time']
    # Combine the seperated dataframes to create a charging sessions 
    ev = pd.concat([df1, df2])
    # Sort the combined dataframes, by time 
    ev = ev.sort_values(by=['time'])
    # Reset dataframe to organized times
    ev = ev.reset_index(drop = True)
    # Adds the 1 and -1s to get number of charging sessions occuring at every time interval
    ev['fuzz'] = ev.value.cumsum()
    # 0 - 4 charging session can occur since there are 4 charging stations
    ev['fuzz'] = ev['fuzz'].clip(lower=0, upper = 4)
    # Multiply by 5 kW which is the power consumption for each EV charger
    ev['charge'] = ev['fuzz'] * 5 
    # Organize columns 
    ev = ev[['time', 'value', 'fuzz', 'charge']]
    return [df, ev]

## Call the function and adjust the parameters

In [5]:
arriv_mean = [0.3, 0.18, 0.12] # Average number of car arrivals
cpd_mean = [8, 9, 13] # Arrival times peaks 
cpd_std_dev = [166, 179, 150] # [332, 358, 300] # Arrival times standard deviation
ev_arriv = np.array([])
start_date = pd.Timestamp(2022, 1, 1, 0) # Start date for the time deltas
charge_mean = [90, 90, 90] # Average length of charging session 
charge_std_dev = [30, 30, 30] # Standard deviation of charging length
num_days = 365 # Number of outputed random data 
# Call the function 
df, ev = date_time_arrival_times(start_date, num_days, arriv_mean, cpd_mean,
                                 cpd_std_dev, charge_mean, charge_std_dev)
# Display dataframe
ev

1116


,time,value,fuzz,charge
0,2022-01-01 07:55:00.000000000,1,1,5
1,2022-01-01 09:08:02.450460840,-1,0,0
2,2022-01-01 10:05:00.000000000,1,1,5
3,2022-01-01 11:26:57.822455760,-1,0,0
4,2022-01-01 16:02:00.000000000,1,1,5
...,...,...,...,...
2227,2022-12-31 07:39:10.110357360,-1,0,0
2228,2022-12-31 08:28:00.000000000,1,1,5
2229,2022-12-31 10:09:35.311610820,-1,0,0
2230,2022-12-31 18:06:00.000000000,1,1,5


In [13]:
arriv = ev[ev.value == 1].reset_index(drop=True)
arriv

,time,value,fuzz,charge
0,2022-01-01 07:55:00,1,1,5
1,2022-01-01 10:05:00,1,1,5
2,2022-01-01 16:02:00,1,1,5
3,2022-01-02 07:22:00,1,1,5
4,2022-01-02 07:28:00,1,2,10
...,...,...,...,...
1111,2022-12-30 03:40:00,1,1,5
1112,2022-12-30 13:54:00,1,1,5
1113,2022-12-31 06:41:00,1,1,5
1114,2022-12-31 08:28:00,1,1,5


In [ ]:
avg_day

## Convert on off sessions into 1 minute data for 1 year 

## Convert on off sessions into 1 minute data for 1 year 

Resampler function

In [6]:
def resample(df, time_column = 'time', resample_rate = '5T'):
    df[time_column] = pd.to_datetime(df[time_column])
    df = df.set_index(df[time_column])
    df = df.drop(columns = [time_column])
    df = df.resample(resample_rate).last().ffill()  #df.groupby(pd.Grouper(key=time_column, freq=resample_rate)).ffill().bfill() 
    df.insert(loc=0, column=time_column, value=df.index) #df.insert(loc=0, column=time_column, value=df.index)
    df = df.reset_index(drop=True)
    #df.fillna(0)
    #df = df.astype(int, errors='ignore')
    #df = df.fillna(0)
    return df

 Insert start and end timestamp to ensure a complete 15 minute year (35040 rows), and call the resampler function

In [14]:
first_row = {'time':pd.Timestamp('2022-01-01 00:00:00'), 'value':0, 'fuzz':0, 'charge':0}
last_row = {'time':pd.Timestamp('2023-01-01 00:00:00'), 'value':0, 'fuzz':0, 'charge':0}
ev_pad = ev
ev_pad = pd.concat([ev_pad, pd.DataFrame([first_row])], ignore_index=True)
ev_pad = pd.concat([ev_pad, pd.DataFrame([last_row])], ignore_index=True)
ev_pad = ev_pad.sort_values(by=['time'])
ev_pad = ev_pad.reset_index(drop = True)
ev_pad = resample(ev_pad, resample_rate = '1T')
ev_pad = ev_pad[ev_pad.time.between('2022-01-01 00:00:00', '2022-12-31 23:59:59')].reset_index(drop=True)
ev_pad

,time,value,fuzz,charge
0,2022-01-01 00:00:00,0.0,0.0,0.0
1,2022-01-01 00:01:00,0.0,0.0,0.0
2,2022-01-01 00:02:00,0.0,0.0,0.0
3,2022-01-01 00:03:00,0.0,0.0,0.0
4,2022-01-01 00:04:00,0.0,0.0,0.0
...,...,...,...,...
525595,2022-12-31 23:55:00,-1.0,0.0,0.0
525596,2022-12-31 23:56:00,-1.0,0.0,0.0
525597,2022-12-31 23:57:00,-1.0,0.0,0.0
525598,2022-12-31 23:58:00,-1.0,0.0,0.0


## Find the average day for the random genrator data and compare it to the original data

In [15]:
time_array, avg_day = time_sep_label(ev_pad, value_column = 'charge')

In [16]:
time_array, avg_day = time_sep_label_sum(arriv, resample_rate = '60T')

In [ ]:
time_array

In [17]:
avg_day = pd.DataFrame({'time': time_array, 'avg_day': avg_day})
avg_day

,time,avg_day
0,00:00,1
1,01:00,2
2,02:00,11
3,03:00,19
4,04:00,41
5,05:00,55
6,06:00,79
7,07:00,122
8,08:00,110
9,09:00,105


In [18]:
lib = pd.read_excel('/home/lencon/Git_Research/summer_2023_conf_paper/Code/EV_Charger/HourlySessionUsage.xlsx')
lib

,hour,sessions
0,0,2
1,1,0
2,2,0
3,3,0
4,4,0
5,5,1
6,6,11
7,7,186
8,8,59
9,9,112


In [23]:
import os
os.chdir('/home/lencon/Git_Research/FISTS_2024/Code/EV_Charger')
plot_graph([avg_day.time] * 2, [lib.sessions, avg_day.avg_day], ['<B>Number of Real Sessions</B>', '<B>Number of Simulated Sessions</B>'], '<B>Level 2 Chargers Number of Sessions in One Year</B>', '<B>Time</B>', '<B>Number of Sessions</B>', 'l2_avg_day_rand_poisson_1_hour')

[19301:0223/153604.236445:ERROR:buffer_manager.cc(488)] [.DisplayCompositor]GL ERROR :GL_INVALID_OPERATION : glBufferData: <- error from previous GL command
[19301:0223/153604.511467:ERROR:buffer_manager.cc(488)] [.DisplayCompositor]GL ERROR :GL_INVALID_OPERATION : glBufferData: <- error from previous GL command


In [ ]:
import os
os.chdir('/home/lencon/Git_Research/summer_2023_conf_paper/Code/EV_Charger')

In [ ]:
#pdf
plot_graph([avg_day.time] * 2, [lib.sessions / lib.sessions.sum(), avg_day.avg_day / avg_day.avg_day.sum()], ['Number of Real Sessions', 'Number of Simulated Sessions'], 'Level 2 Chargers Number of Sessions in One Year', 'Time', 'Probability of a Charging Event', 'l2_avg_day_rand_poisson_1_hour_pdf')

## Plot the power output of the data

In [ ]:
plot_graph([ev_pad.time], [ev_pad.charge], "Level 2 Gaussian Random Generated Data", "Level 2 Gaussian Random Generated Data Run 4", "Time", "Power (kW)", "l2_g_pad_poisson")

In [ ]:
# June
plot_graph([ev_pad.time[ev_pad.time.between('06-01-2022', '07-01-2022')]], [ev_pad.charge[ev_pad.time.between('06-01-2022', '07-01-2022')]], "Level EV Chargers<br/>Power Output<br/>From: June 01, 2022 to July 01, 2022", "Level 2", "Time", "Power (kW)", "l2_g_pad_poisson_June")

In [ ]:
def testPoissonArrivL2(time):
    import numpy as np
    import pandas as pd
    import os


    # Creates the Poisson for number of random arrivals for a day (Scalar value)
    def number_of_arrivals(mean, size =1):
        num_arriv = np.array([])
        for i in range(len(mean)):
            num_arriv = np.append(num_arriv, np.random.poisson(mean[i], size))
        num_arriv = np.sort(np.absolute(num_arriv))
        return (num_arriv)
    # Creates a compund Gaussian with user imputed size (vector length) 
    # A vector is expected for for the function fir both the mean and std dev
    # [1 ... N] creates an Nth compound Gaussian eg, mean = [10, 20, 30], std_dev = [1, 2, 3]  creates a tri-Gaussian
    def compound_gauss(mean, std_dev, size):
        if len(mean) == len(std_dev):
            cpd_gauss = np.array([])
            for i in range(len(mean)):
                if size[i] <= 1:
                    size[i] = 1
                cpd_gauss = np.append(cpd_gauss, np.random.default_rng().normal(mean[i], std_dev[i], int(size[i])))
            cpd_gauss = np.sort(cpd_gauss)
        return (cpd_gauss)
    def compound_poisson(mean, size):
        cpd_poisson = np.array([])
        for i in range(len(mean)):
            if size[i] <= 1:
                size[i] = 1
            cpd_poisson = np.append(cpd_poisson, np.random.poisson(mean[i], int(size[i])) * 60  + np.random.randint(-30, high=30, size = int(size[i])))
        cpd_poisson = np.sort(cpd_poisson)
        return (cpd_poisson)
    # Creates the Poisson for arrival and leave times in minutes in a day
    def random_ev_arrivals_times(arriv_mean, cpd_mean, cpd_std_dev, charge_mean, charge_std_dev):
        # Size (vector length) is determined by number_of_arrivals function
        size = number_of_arrivals(arriv_mean)
        # Input the time when peaks occur
        arriv_time = compound_poisson(cpd_mean, size)
        # Input the average duration of charging for each peak 
        leave_time = compound_gauss(charge_mean, charge_std_dev, size)
        return [arriv_time, leave_time]
    # Creates a datafrrame with the arrival times, and adds a time column 
    # If multiple charging sessions occur they add up
    def date_time_arrival_times(start_date, num_days, arriv_mean, cpd_mean, cpd_std_dev, charge_mean, charge_std_dev):
        # Create a dataframe to be populated 
        df = pd.DataFrame(columns = ['arriv_time', 'leave_time'])
        # Timestamp for each day counter 
        current_date = start_date
        # Populates the dataframe with random arrival for each day
        # Each day is added one at a time in the for loop
        for i in range(num_days):
            # Create temporary dataframe for that day that will be added main dataframe
            temp = pd.DataFrame(columns = ['arriv_time'])
            # Call the random arrival time function for each day
            arriv_time, leave_time = random_ev_arrivals_times(arriv_mean, cpd_mean, cpd_std_dev, charge_mean, charge_std_dev)
            # Converts random arrival times (minutes in a day) into timedeltas which are added to that day's timestamp
            # Converts random arrival times into pandas timestamps 
            temp['arriv_time'] = current_date + pd.to_timedelta(arriv_time, unit = 'min')
            # Converts random leave times (duration) into timestamps by adding arrival time to leave time timedeltas
            temp['leave_time'] = temp['arriv_time'] + pd.to_timedelta(leave_time, unit = 'min')
            # Add daily dataframe to the main dataframe
            df = pd.concat([df,temp])
            # Update the current day timestamp by adding one day
            current_date = current_date + pd.Timedelta(1, "d")
        # Ensures that arrival times are in order
        df = df.sort_values(by=['arriv_time'])
        # Reset dataframe to organized arrival times
        df = df.reset_index(drop = True)
        # Seperate arrival times and leave times to create charging sessions 
        df1 = df[['arriv_time']]
        # Set a charging session to 1
        df1['value'] = 1
        df1['time'] = df1['arriv_time']
        df2 = df[['leave_time']]
        # Set a charging session to -1 to negate a charging to shut it off
        df2['value'] = -1
        df2['time'] = df2['leave_time']
        # Combine the seperated dataframes to create a charging sessions 
        ev = pd.concat([df1, df2])
        # Sort the combined dataframes, by time 
        ev = ev.sort_values(by=['time'])
        # Reset dataframe to organized times
        ev = ev.reset_index(drop = True)
        # Adds the 1 and -1s to get number of charging sessions occuring at every time interval
        ev['fuzz'] = ev.value.cumsum()
        # 0 - 4 charging session can occur since there are 4 charging stations
        ev['fuzz'] = ev['fuzz'].clip(lower=0, upper = 4)
        # Multiply by 5 kW which is the power consumption for each EV charger
        ev['charge'] = ev['fuzz'] * 5 
        # Organize columns 
        ev = ev[['time', 'value', 'fuzz', 'charge']]
        return [df, ev]
        
    def resample(df, time_column = 'time', resample_rate = '5T'):
        df[time_column] = pd.to_datetime(df[time_column])
        df = df.set_index(df[time_column])
        df = df.drop(columns = [time_column])
        df = df.resample(resample_rate).last().ffill()  #df.groupby(pd.Grouper(key=time_column, freq=resample_rate)).ffill().bfill() 
        df.insert(loc=0, column=time_column, value=df.index) #df.insert(loc=0, column=time_column, value=df.index)
        df = df.reset_index(drop=True)
        #df.fillna(0)
        #df = df.astype(int, errors='ignore')
        #df = df.fillna(0)
        return df
    
    temporaryFile = "df_l2_.feather"  
    if os.path.exists(temporaryFile):
        df = pd.read_feather(temporaryFile)
        index = time / 900
        output = df.charge[index].item()
        return output
    else: 
        arriv_mean = [6, 2, 1] # Average number of car arrivals
        cpd_mean = [7, 9, 13] # Arrival times peaks 
        cpd_std_dev = [166, 179, 150] # [332, 358, 300] # Arrival times standard deviation
        ev_arriv = np.array([])
        start_date = pd.Timestamp(2023, 1, 1, 0) # Start date for the time deltas
        charge_mean = [90, 90, 90] # Average length of charging session 
        charge_std_dev = [30, 30, 30] # Standard deviation of charging length
        num_days = 365 # Number of outputed random data 
        # Call the function 
        df, ev = date_time_arrival_times(start_date, num_days, arriv_mean, cpd_mean,
                                 cpd_std_dev, charge_mean, charge_std_dev)
        first_row = {'time':pd.Timestamp('2023-01-01 00:00:00'), 'value':0, 'fuzz':0, 'charge':0}
        last_row = {'time':pd.Timestamp('2024-01-01 00:00:00'), 'value':0, 'fuzz':0, 'charge':0}
        ev_pad = ev
        ev_pad = pd.concat([ev_pad, pd.DataFrame([first_row])], ignore_index=True)
        ev_pad = pd.concat([ev_pad, pd.DataFrame([last_row])], ignore_index=True)
        ev_pad = ev_pad.sort_values(by=['time'])
        ev_pad = ev_pad.reset_index(drop = True)
        ev_pad = resample(ev_pad, resample_rate = '15T')
        ev_pad = ev_pad[ev_pad.time.between('2023-01-01 00:00:00', '2023-12-31 23:59:59')].reset_index(drop=True)
        ev_pad.to_feather(temporaryFile)
        return ev_pad.charge[0].item()

In [ ]:
def testPoissonArrivL2(time):
    import numpy as np
    import pandas as pd
    import os

    np.random.seed(10)
    # Creates the Poisson for number of random arrivals for a day (Scalar value)
    def number_of_arrivals(mean, size =1):
        num_arriv = np.array([])
        for i in range(len(mean)):
            num_arriv = np.append(num_arriv, np.random.poisson(mean[i], size))
        num_arriv = np.sort(np.absolute(num_arriv))
        return (num_arriv)
    # Creates a compund Gaussian with user imputed size (vector length) 
    # A vector is expected for for the function fir both the mean and std dev
    # [1 ... N] creates an Nth compound Gaussian eg, mean = [10, 20, 30], std_dev = [1, 2, 3]  creates a tri-Gaussian
    def compound_gauss(mean, std_dev, size):
        if len(mean) == len(std_dev):
            cpd_gauss = np.array([])
            for i in range(len(mean)):
                if size[i] <= 1:
                    size[i] = 1
                cpd_gauss = np.append(cpd_gauss, np.random.default_rng().normal(mean[i], std_dev[i], int(size[i])))
            cpd_gauss = np.sort(cpd_gauss)
        return (cpd_gauss)
    def compound_poisson(mean, size):
        cpd_poisson = np.array([])
        for i in range(len(mean)):
            if size[i] <= 1:
                size[i] = 1
            cpd_poisson = np.append(cpd_poisson, np.random.poisson(mean[i], int(size[i])) * 60  + np.random.randint(-30, high=30, size = int(size[i])))
        cpd_poisson = np.sort(cpd_poisson)
        return (cpd_poisson)
    # Creates the Poisson for arrival and leave times in minutes in a day
    def random_ev_arrivals_times(arriv_mean, cpd_mean, cpd_std_dev, charge_mean, charge_std_dev):
        # Size (vector length) is determined by number_of_arrivals function
        size = number_of_arrivals(arriv_mean)
        # Input the time when peaks occur
        arriv_time = compound_poisson(cpd_mean, size)
        # Input the average duration of charging for each peak 
        leave_time = compound_gauss(charge_mean, charge_std_dev, size)
        return [arriv_time, leave_time]
    # Creates a datafrrame with the arrival times, and adds a time column 
    # If multiple charging sessions occur they add up
    def date_time_arrival_times(start_date, num_days, arriv_mean, cpd_mean, cpd_std_dev, charge_mean, charge_std_dev):
        # Create a dataframe to be populated 
        df = pd.DataFrame(columns = ['arriv_time', 'leave_time'])
        # Timestamp for each day counter 
        current_date = start_date
        # Populates the dataframe with random arrival for each day
        # Each day is added one at a time in the for loop
        for i in range(num_days):
            # Create temporary dataframe for that day that will be added main dataframe
            temp = pd.DataFrame(columns = ['arriv_time'])
            # Call the random arrival time function for each day
            arriv_time, leave_time = random_ev_arrivals_times(arriv_mean, cpd_mean, cpd_std_dev, charge_mean, charge_std_dev)
            # Converts random arrival times (minutes in a day) into timedeltas which are added to that day's timestamp
            # Converts random arrival times into pandas timestamps 
            temp['arriv_time'] = current_date + pd.to_timedelta(arriv_time, unit = 'min')
            # Converts random leave times (duration) into timestamps by adding arrival time to leave time timedeltas
            temp['leave_time'] = temp['arriv_time'] + pd.to_timedelta(leave_time, unit = 'min')
            # Add daily dataframe to the main dataframe
            df = pd.concat([df,temp])
            # Update the current day timestamp by adding one day
            current_date = current_date + pd.Timedelta(1, "d")
        # Ensures that arrival times are in order
        df = df.sort_values(by=['arriv_time'])
        # Reset dataframe to organized arrival times
        df = df.reset_index(drop = True)
        # Seperate arrival times and leave times to create charging sessions 
        df1 = df[['arriv_time']]
        # Set a charging session to 1
        df1['value'] = 1
        df1['time'] = df1['arriv_time']
        df2 = df[['leave_time']]
        # Set a charging session to -1 to negate a charging to shut it off
        df2['value'] = -1
        df2['time'] = df2['leave_time']
        # Combine the seperated dataframes to create a charging sessions 
        ev = pd.concat([df1, df2])
        # Sort the combined dataframes, by time 
        ev = ev.sort_values(by=['time'])
        # Reset dataframe to organized times
        ev = ev.reset_index(drop = True)
        # Adds the 1 and -1s to get number of charging sessions occuring at every time interval
        ev['fuzz'] = ev.value.cumsum()
        # 0 - 4 charging session can occur since there are 4 charging stations
        ev['fuzz'] = ev['fuzz'].clip(lower=0, upper = 4)
        # Multiply by 5 kW which is the power consumption for each EV charger
        ev['charge'] = ev['fuzz'] * 5000
        # Organize columns 
        ev = ev[['time', 'value', 'fuzz', 'charge']]
        return [df, ev]
        
    def resample(df, time_column = 'time', resample_rate = '5T'):
        df[time_column] = pd.to_datetime(df[time_column])
        df = df.set_index(df[time_column])
        df = df.drop(columns = [time_column])
        df = df.resample(resample_rate).last().ffill()  #df.groupby(pd.Grouper(key=time_column, freq=resample_rate)).ffill().bfill() 
        df.insert(loc=0, column=time_column, value=df.index) #df.insert(loc=0, column=time_column, value=df.index)
        df = df.reset_index(drop=True)
        #df.fillna(0)
        #df = df.astype(int, errors='ignore')
        #df = df.fillna(0)
        return df
    
    temporaryFile = "df_l2_.feather"  
    if os.path.exists(temporaryFile):
        df = pd.read_feather(temporaryFile)
        index = time / 900
        output = df.charge[index].item()
        return output
    else: 
        arriv_mean = [6, 2, 1] # Average number of car arrivals
        cpd_mean = [7, 9, 13] # Arrival times peaks 
        cpd_std_dev = [166, 179, 150] # [332, 358, 300] # Arrival times standard deviation
        ev_arriv = np.array([])
        start_date = pd.Timestamp(2023, 1, 1, 0) # Start date for the time deltas
        charge_mean = [90, 90, 90] # Average length of charging session 
        charge_std_dev = [30, 30, 30] # Standard deviation of charging length
        num_days = 365 # Number of outputed random data 
        # Call the function 
        df, ev = date_time_arrival_times(start_date, num_days, arriv_mean, cpd_mean,
                                 cpd_std_dev, charge_mean, charge_std_dev)
        first_row = {'time':pd.Timestamp('2023-01-01 00:00:00'), 'value':0, 'fuzz':0, 'charge':0}
        last_row = {'time':pd.Timestamp('2024-01-01 00:00:00'), 'value':0, 'fuzz':0, 'charge':0}
        ev_pad = ev
        ev_pad = pd.concat([ev_pad, pd.DataFrame([first_row])], ignore_index=True)
        ev_pad = pd.concat([ev_pad, pd.DataFrame([last_row])], ignore_index=True)
        ev_pad = ev_pad.sort_values(by=['time'])
        ev_pad = ev_pad.reset_index(drop = True)
        ev_pad = resample(ev_pad, resample_rate = '15T')
        ev_pad = ev_pad[ev_pad.time.between('2023-01-01 00:00:00', '2023-12-31 23:59:59')].reset_index(drop=True)
        ev_pad['charge'] *= -1
        ev_pad.to_feather(temporaryFile)
        return ev_pad.charge[0].item()

In [ ]:
def testPoissonArrivL3(time):
    import numpy as np
    import pandas as pd
    import os

    np.random.seed(10)
    # Creates the Poisson for number of random arrivals for a day (Scalar value)
    def number_of_arrivals(mean, size =1):
        num_arriv = np.array([])
        for i in range(len(mean)):
            num_arriv = np.append(num_arriv, np.random.poisson(mean[i], size))
        num_arriv = np.sort(np.absolute(num_arriv))
        return (num_arriv)
    # Creates a compund Gaussian with user imputed size (vector length) 
    # A vector is expected for for the function fir both the mean and std dev
    # [1 ... N] creates an Nth compound Gaussian eg, mean = [10, 20, 30], std_dev = [1, 2, 3]  creates a tri-Gaussian
    def compound_gauss(mean, std_dev, size):
        if len(mean) == len(std_dev):
            cpd_gauss = np.array([])
            for i in range(len(mean)):
                if size[i] <= 1:
                    size[i] = 1
                cpd_gauss = np.append(cpd_gauss, np.random.default_rng().normal(mean[i], std_dev[i], int(size[i])))
            cpd_gauss = np.sort(cpd_gauss)
        return (cpd_gauss)
    def compound_poisson(mean, size):
        cpd_poisson = np.array([])
        for i in range(len(mean)):
            if size[i] <= 1:
                size[i] = 1
            cpd_poisson = np.append(cpd_poisson, np.random.poisson(mean[i], int(size[i])) * 60  + np.random.randint(-30, high=30, size = int(size[i])))
        cpd_poisson = np.sort(cpd_poisson)
        return (cpd_poisson)
    # Creates the Poisson for arrival and leave times in minutes in a day
    def random_ev_arrivals_times(arriv_mean, cpd_mean, cpd_std_dev, charge_mean, charge_std_dev):
        # Size (vector length) is determined by number_of_arrivals function
        size = number_of_arrivals(arriv_mean)
        # Input the time when peaks occur
        arriv_time = compound_poisson(cpd_mean, size)
        # Input the average duration of charging for each peak 
        leave_time = compound_gauss(charge_mean, charge_std_dev, size)
        return [arriv_time, leave_time]
    # Creates a datafrrame with the arrival times, and adds a time column 
    # If multiple charging sessions occur they add up
    def date_time_arrival_times(start_date, num_days, arriv_mean, cpd_mean, cpd_std_dev, charge_mean, charge_std_dev):
        # Create a dataframe to be populated 
        df = pd.DataFrame(columns = ['arriv_time', 'leave_time'])
        # Timestamp for each day counter 
        current_date = start_date
        # Populates the dataframe with random arrival for each day
        # Each day is added one at a time in the for loop
        for i in range(num_days):
            # Create temporary dataframe for that day that will be added main dataframe
            temp = pd.DataFrame(columns = ['arriv_time'])
            # Call the random arrival time function for each day
            arriv_time, leave_time = random_ev_arrivals_times(arriv_mean, cpd_mean, cpd_std_dev, charge_mean, charge_std_dev)
            # Converts random arrival times (minutes in a day) into timedeltas which are added to that day's timestamp
            # Converts random arrival times into pandas timestamps 
            temp['arriv_time'] = current_date + pd.to_timedelta(arriv_time, unit = 'min')
            # Converts random leave times (duration) into timestamps by adding arrival time to leave time timedeltas
            temp['leave_time'] = temp['arriv_time'] + pd.to_timedelta(leave_time, unit = 'min')
            # Add daily dataframe to the main dataframe
            df = pd.concat([df,temp])
            # Update the current day timestamp by adding one day
            current_date = current_date + pd.Timedelta(1, "d")
        # Ensures that arrival times are in order
        df = df.sort_values(by=['arriv_time'])
        # Reset dataframe to organized arrival times
        df = df.reset_index(drop = True)
        # Seperate arrival times and leave times to create charging sessions 
        df1 = df[['arriv_time']]
        # Set a charging session to 1
        df1['value'] = 1
        df1['time'] = df1['arriv_time']
        df2 = df[['leave_time']]
        # Set a charging session to -1 to negate a charging to shut it off
        df2['value'] = -1
        df2['time'] = df2['leave_time']
        # Combine the seperated dataframes to create a charging sessions 
        ev = pd.concat([df1, df2])
        # Sort the combined dataframes, by time 
        ev = ev.sort_values(by=['time'])
        # Reset dataframe to organized times
        ev = ev.reset_index(drop = True)
        # Adds the 1 and -1s to get number of charging sessions occuring at every time interval
        ev['fuzz'] = ev.value.cumsum()
        # 0 - 4 charging session can occur since there are 4 charging stations
        ev['fuzz'] = ev['fuzz'].clip(lower=0, upper = 4)
        # Multiply by 5 kW which is the power consumption for each EV charger
        ev['charge'] = ev['fuzz'] * 5000 
        # Organize columns 
        ev = ev[['time', 'value', 'fuzz', 'charge']]
        return [df, ev]
        
    def resample(df, time_column = 'time', resample_rate = '5T'):
        df[time_column] = pd.to_datetime(df[time_column])
        df = df.set_index(df[time_column])
        df = df.drop(columns = [time_column])
        df = df.resample(resample_rate).last().ffill()  #df.groupby(pd.Grouper(key=time_column, freq=resample_rate)).ffill().bfill() 
        df.insert(loc=0, column=time_column, value=df.index) #df.insert(loc=0, column=time_column, value=df.index)
        df = df.reset_index(drop=True)
        #df.fillna(0)
        #df = df.astype(int, errors='ignore')
        #df = df.fillna(0)
        return df
    
    temporaryFile = "df_l3_.feather"  
    if os.path.exists(temporaryFile):
        df = pd.read_feather(temporaryFile)
        index = time / 900
        output = df.charge[index].item()
        return output
    else: 
        arriv_mean = [2, 1, 1] # Average number of car arrivals
        cpd_mean = [7, 9, 13] # Arrival times peaks 
        cpd_std_dev = [166, 179, 150] # [332, 358, 300] # Arrival times standard deviation
        ev_arriv = np.array([])
        start_date = pd.Timestamp(2023, 1, 1, 0) # Start date for the time deltas
        charge_mean = [30, 30, 30] # Average length of charging session 
        charge_std_dev = [10, 10, 10] # Standard deviation of charging length
        num_days = 365 # Number of outputed random data 
        # Call the function 
        df, ev = date_time_arrival_times(start_date, num_days, arriv_mean, cpd_mean,
                                 cpd_std_dev, charge_mean, charge_std_dev)
        first_row = {'time':pd.Timestamp('2023-01-01 00:00:00'), 'value':0, 'fuzz':0, 'charge':0}
        last_row = {'time':pd.Timestamp('2024-01-01 00:00:00'), 'value':0, 'fuzz':0, 'charge':0}
        ev_pad = ev
        ev_pad = pd.concat([ev_pad, pd.DataFrame([first_row])], ignore_index=True)
        ev_pad = pd.concat([ev_pad, pd.DataFrame([last_row])], ignore_index=True)
        ev_pad = ev_pad.sort_values(by=['time'])
        ev_pad = ev_pad.reset_index(drop = True)
        ev_pad = resample(ev_pad, resample_rate = '15T')
        ev_pad = ev_pad[ev_pad.time.between('2023-01-01 00:00:00', '2023-12-31 23:59:59')].reset_index(drop=True)
        ev_pad['charge'] *= -1
        ev_pad.to_feather(temporaryFile)
        return ev_pad.charge[0].item()

In [ ]:
testPoissonArrivL2(27000000)

In [ ]:
import pandas as pd
df = pd.read_feather('df_l2_.feather')
df

In [ ]:
df.plot(x = 'time', y = 'charge')